In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [29]:
#import the csv
df = pd.read_csv("wcb.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [44]:
#Cleaning the dataset + preprocessing
l = LabelEncoder()
l.fit(df['diagnosis'])
df['diagnosis'] = l.transform(df['diagnosis'])

X = df.drop(['id', 'diagnosis', 'Unnamed: 32'], axis=1)
Y = df['diagnosis'].values
Xnames = X.columns

In [45]:
#Reducing multicollinearity
corrMat = X.corr().values
for i in range(corrMat.shape[0]):
    for j in range(corrMat.shape[1]):
        if abs(corrMat[i][j]) > 0.7 and i != j:
            print(Xnames[i], Xnames[j], corrMat[i][j])

radius_mean perimeter_mean 0.9978552814938109
radius_mean area_mean 0.9873571700566127
radius_mean concave points_mean 0.8225285223871786
radius_mean area_se 0.7358636632128178
radius_mean radius_worst 0.9695389726112055
radius_mean perimeter_worst 0.9651365139559871
radius_mean area_worst 0.9410824595860461
radius_mean concave points_worst 0.7442141978674661
texture_mean texture_worst 0.912044588840421
perimeter_mean radius_mean 0.9978552814938109
perimeter_mean area_mean 0.9865068039913907
perimeter_mean concavity_mean 0.7161356501174047
perimeter_mean concave points_mean 0.8509770412528191
perimeter_mean area_se 0.7449826941190683
perimeter_mean radius_worst 0.9694763634663146
perimeter_mean perimeter_worst 0.9703868870426394
perimeter_mean area_worst 0.9415498080023068
perimeter_mean concave points_worst 0.7712407893605497
area_mean radius_mean 0.9873571700566127
area_mean perimeter_mean 0.9865068039913907
area_mean concave points_mean 0.8232688686524342
area_mean radius_se 0.73256